In [8]:
# %%
from dotenv import load_dotenv
import os
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.tools import TavilySearchResults
from huggingface_hub import InferenceClient
from langgraph.graph import Graph
from langchain_core.language_models import LLM

# Load environment variables
load_dotenv()
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
HUGGING_API_KEY = os.getenv("HUGGING_API_KEY")

# Initialize API Clients
tavily_search = TavilySearchResults(api_key=TAVILY_API_KEY)
client = InferenceClient(token=HUGGING_API_KEY)


In [9]:

# Custom LLM Wrapper for InferenceClient
class HuggingFaceLLM(LLM):
    model: str = "google/flan-t5-large"

    def _call(self, prompt, stop=None):
        return client.text_generation(prompt, model=self.model, max_new_tokens=100)

    @property
    def _llm_type(self):
        return "huggingface_inference"

# Initialize LLM
llm = HuggingFaceLLM()

# Research Agent using LLMChain
research_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Summarize the following topic in 20 words:\n{topic}"
)
research_chain = LLMChain(llm=llm, prompt=research_prompt)

# Answer Drafter Agent using LLMChain
draft_prompt = PromptTemplate(
    input_variables=["summary"],
    template="Draft a detailed and structured answer based on the following summary:\n{summary}"
)
draft_chain = LLMChain(llm=llm, prompt=draft_prompt)

# Research Agent
def research_agent(query):
    search_result = tavily_search.run(query)
    if not search_result or not isinstance(search_result, list):
        return "No relevant search results found."

    formatted_data = "\n".join(
        [f"- {item.get('content', '')[:300]}..." for item in search_result if 'content' in item]
    )

    if not formatted_data.strip():
        return "No relevant data found in search results."

    return research_chain.run(topic=formatted_data)

# Answer Drafter Agent
def answer_drafter_agent(summary):
    if summary in ["No relevant search results found.", "No relevant data found in search results."]:
        return summary
    return draft_chain.run(summary=summary)


In [10]:

# LangGraph Workflow
def research_node(state):
    return {"summary": research_agent(state["query"])}

def draft_node(state):
    return {"answer": answer_drafter_agent(state["summary"])}

# Create workflow graph
workflow = Graph()
workflow.add_node("research", research_node)
workflow.add_node("draft", draft_node)

# Connect nodes
workflow.add_edge("research", "draft")
workflow.set_entry_point("research")
workflow.set_finish_point("draft")

# Compile and execute workflow
app = workflow.compile()



In [11]:

# Answer Drafter Agent
def answer_drafter_agent(summary):
    if summary in ["No relevant search results found.", "No relevant data found in search results."]:
        return summary
    return draft_chain.run(summary=summary)

# LangGraph Workflow
def research_node(state):
    return {"summary": research_agent(state["query"])}

def draft_node(state):
    return {"answer": answer_drafter_agent(state["summary"])}

# Create workflow graph
workflow = Graph()
workflow.add_node("research", research_node)
workflow.add_node("draft", draft_node)

# Connect nodes
workflow.add_edge("research", "draft")
workflow.set_entry_point("research")
workflow.set_finish_point("draft")

# Compile and execute workflow
app = workflow.compile()

# Example Execution
query = "Latest advancements in makeup research"
result = app.invoke({"query": query})
print("Final Answer:", result["answer"])


Final Answer: The latest innovations in cosmetics are a reflection of the changing face of the industry. The latest innovations in cosmetics are a reflection of the changing face of the industry. The latest innovations in cosmetics are a reflection of the changing face of the industry.
